In [76]:
def read_file(file):
    rules = []
    atualizations = []
    flag = False
    with open(file, 'r') as f:
        for line in f:
            if line == '\n':
                flag = True
                continue

            if not flag:
                rules.append(line.strip())
            else:
                atualizations.append(line.strip())
                
    return rules, atualizations

rules, atualizations = read_file('inputs/5.txt')
len(rules), len(atualizations)

(1176, 175)

In [77]:
print(rules[:5])
print(atualizations[:5])

['45|19', '55|17', '55|38', '58|41', '58|46']
['56,79,55,52,85,41,61,97,64,72,86,46,58,48,96,62,76,12,13', '16,95,77,51,17,49,33,56,97', '95,38,49,61,32,19,77,22,13,27,56', '84,22,89,64,45,79,34,85,72,96,48,55,29', '84,34,18,93,63,16,95,77,51,32,17,26,54,27,44,11,19,42,28,38,49,33,56']


# Part 1

In [78]:
# Processar as regras para pares (A, B)
pares = [tuple(map(int, regra.split('|'))) for regra in rules]

In [79]:
pares[:5]

[(45, 19), (55, 17), (55, 38), (58, 41), (58, 46)]

In [80]:
accumulated = 0
for i, lista in enumerate(atualizations):
    numeros = list(map(int, lista.split(',')))
    sequence_valid = True
    for a, b in pares:
        if a in numeros and b in numeros:
            pos_a = numeros.index(a)
            pos_b = numeros.index(b)
            if pos_a >= pos_b:  # Regra violada
                sequence_valid = False
                break
    if sequence_valid:
        # somar o valor do meio ao acumulado
        mid = len(numeros)//2
        accumulated += numeros[mid]
        # break
    

In [81]:
accumulated

4185

# Part 2

In [ ]:
from collections import defaultdict, deque

# grafos para as dependências
pages_before = defaultdict(set)
pages_after = defaultdict(set)

for a, b in pares:
    pages_before[b].add(a)
    pages_after[a].add(b)


accumulated = 0

# verifica se a sequência respeita as regras de precedência
def is_sequence_valid(sequence, pages_before): 
    for i, x in enumerate(sequence):
        for j, y in enumerate(sequence):
            if x in pages_before[y] and i > j:
                return False
    return True

# reorganiza a sequência e torná-la válida
def make_sequence_valid(sequence, pages_before, pages_after):
    ordered_sequence = []
    queue = deque([])
    
    remaining_dependencies = {v: len(pages_before[v] & set(sequence)) for v in sequence}
    for v in sequence:
        if remaining_dependencies[v] == 0: # adicionar as páginas sem dependências
            queue.append(v)
    
    # organizar as páginas respeitando as dependências
    while queue:
        x = queue.popleft()
        ordered_sequence.append(x)
        for y in pages_after[x]:
            if y in remaining_dependencies:
                remaining_dependencies[y] -= 1
                if remaining_dependencies[y] == 0:
                    queue.append(y)
    
    return ordered_sequence


for atualization in atualizations:
    atualization = list(map(int, atualization.split(',')))
    sequence_valid = is_sequence_valid(atualization, pages_before)
    if not sequence_valid:
        ordered_sequence = make_sequence_valid(atualization, pages_before, pages_after)
        accumulated += ordered_sequence[len(ordered_sequence) // 2]
print(accumulated)


4480
